In [1]:
# Voer dit blok code uit door met de cursor in het blok te klikken, en vervolgens SHIFT+ENTER in te drukken.

import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import odeint

from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout


%matplotlib inline

style = {'description_width': '200px'}
slider_layout = Layout(width='50%')

# T3-Afgeleiden
### Introductie

Het begrip *afgeleide* komt frequent voor in allerlei wetenschappelijke en andere contexten. De techniek van afgeleiden laat toe om erg precies, erg efficiënt en in erg uiteenlopende situaties *veranderingen* te bestuderen. 

Vele fenomenen zoals bewegingen, temperatuursverlopen, productievolumes, epidemies en hoogteprofielen van wielerwedstrijden kunnen beschreven worden als functies. *Afgeleiden* zeggen dan iets over de *veranderingen van de functiewaarden*. Hoe *verandert* de temperatuur, het geproduceerde volume, het aantal besmette personen als de tijd, de prijs of de sociale contacten veranderen?

In deze context begrijpen we functies dikwijls als een verband tussen een 'onafhankelijke' variabele (de tijd, de prijs, de lockdown maatregelen) en de 'afhankelijke' variabele (de temperatuur, de hoeveelheid geproduceerde producten of het aantal besmette personen). In wiskundige taal noemen we de onafhankelijke variabele typisch $x$ (maar soms ook $t$ als het over tijd gaat, of $s$, of andere letters) en de afhankelijke grootheid is dan een functie die we typisch $f$ noemen (maar soms ook $T$ als het over temperatuur gaat, of $v$ en $a$ als het snelheden en versnellingen zijn).

Afgeleiden geven informatie over de *verhouding* tussen *de verandering van de afhankelijke variabele* ten opzichte van *de verandering van de onafhankelijke variabele*. Als je de onafhankelijke variabele varieert, wat gebeurt er dan met de afhankelijke variabele? Stijgt die, of daalt die, en hoe snel of hoe traag gebeurt dat? 
Je kan hetzelfde ook begrijpen in termen van *gevoeligheid*: hoe gevoelig is de afhankelijke variabele aan een wijziging in de onafhankelijke variabele? 

### Een inleidend voorbeeld

We illustreren eerst aan de hand van een voorbeeld wat we bedoelen met *veranderingen* van functiewaarden, en *verhoudingen* tussen *de verandering van de afhankelijke variabele* ten opzichte van *de verandering van de onafhankelijke variabele*.

In een punt $a+h$ in de buurt van $a$ is de functiewaarde van $f$ gelijk aan $f(a+h)$.
    We berekenen voor enkele steeds kleiner wordende $h$ telkens de functiewaarden in $a$ en in $a+h$ en verandering $f(a+h)-f(a)$ die hoort bij deze $h$.
    Zoals te verwachten was, wordt de verandering $f(a+h)-f(a)$ ook erg klein zodra $h$ klein wordt. Maar het is erg interessant op te merken dat de *verhouding* $\frac{f(a+h)-f(a)}{h}$ *niet* noodzakelijk klein wordt!

In [51]:
x = np.linspace(-1,2,100)

a=1

def animatie(h):


    def f(x):
        return x**2+1

    plt.plot(x, f(x), 'r',label='$f(x)=x^2+1$')

    plt.plot([a,a+h], [f(a),f(a)], 'k')
    plt.text(a+h/2, f(a)-0.2, 'h')

    plt.plot([a+h,a+h], [f(a),f(a+h)], 'k')
    plt.text(a+h+0.1,f(a)+(f(a+h)-f(a))/2, 'f(a+h)-f(a)')
    
    plt.plot([a,a+h], [f(a),f(a+h)], 'k')


    plt.title(r'$a={}$, $h={}$, $(f(a+h) - f(a))/h ={:.2f} $'.format(a,h,(f(a+h)-f(a))/h))
    plt.legend()

    
interact(animatie, h=FloatSlider(min=0.1, max=1, step=0.1, value=0.5, description='h', style=style, layout=slider_layout));

interactive(children=(FloatSlider(value=0.5, description='h', layout=Layout(width='50%'), max=1.0, min=0.1, st…

 | a |   h   |  a+h  | f(a) | f(a+h)  | f(a+h)-f(a) |f(a+h)-f(a)/h   |
 | --- | --- | --- | --- | ---| --- | --- |
		|	1 |   1   |   2   |  2  |   5     |      3      | 3       |
		|	1 |  0.5  |  1.5  |  2  |  3.25  |    1.25     | 2.5      |
		|	1 |  0.1  |  1.1  |  2  |  2.21  |    0.21     | 2.1      |
		|	1 | 0.01  | 1.01  |  2  | 2.0201 |  0.00201   | 2.01      |
		|	1 | 0.001 | 1.001 |  2  | 2.0020  |  0.00020   | 2.001     |
        
        
        
 | a |   h   |  a+h  | f(a) | f(a+h)  | f(a+h)-f(a) |f(a+h)-f(a)/h   |
 | --- | --- | --- | --- | ---| --- | --- |
			|3 |   1   |   4   |  10  |   17    |      7     |            7    |
			|3 |  0.5  |  3.5  |  10  |  13.25  |    3.25    |           6.5   |
			|3 |  0.1  |  3.1  |  10  |  10.61  |    0.61    |           6.1   |
			|3 | 0.01  | 3.01  |  10  | 10.0601 |   0.0601   |          6.01   |
			|3 | 0.001 | 3.001 |  10  | 10.006  |  0.006001  |          6.001  |

### Definitie

We noemen een reële functie $f$ *afleidbaar* of *differentieerbaar* in een punt $a$ van haar domein	
    als de limiet 
	$
	\lim_{h\to0} \dfrac{f(a+h)-f(a)}{h}
	$
*bestaat* en *eindig* is.
    
In dat geval noemen we die limiet de **afgeleide van $f$ in $a$** en we noteren
	$$
	f'(a) \equiv \lim_{h\to0} \dfrac{f(a+h)-f(a)}{h}.
	$$

We berekenen nu als voorbeeld enkele afgeleiden van eenvoudige functies rechtstreeks op basis van deze definitie. In de praktijk zal je deze afgeleiden echter altijd met rekenregels berekenen.

Bereken de afgeleide van $f(x) = x^2+1$ in het punt $a=3$:

$$
\begin{align}
f'(3) & = \underset {h \to  0} \lim \dfrac{f(3+h)-f(3)}{h} \\
      & = \underset {h \to  0} \lim \dfrac{(3+h)^2+1-(3^2+1)}{h} \\
      & = \underset {h \to  0} \lim \dfrac{9+6h+h^2 + 1 - 9 -1}{h} \\
      & = \underset {h \to  0} \lim \dfrac{6h+h^2}{h} = \underset {h \to  0} \lim (6+h) = 6
\end{align}
$$

Bereken de afgeleide van $f(x) = x^2+1$ in een willekeurig punt $a$:

$$
\begin{align}
f'(a) & = \underset {h \to  0} \lim \dfrac{f(a+h)-f(a)}{h} \\
      & = \underset {h \to  0} \lim \dfrac{(a+h)^2+1-(a^2+1)}{h} \\
      & = \underset {h \to  0} \lim \dfrac{a^2+2ah+h^2 + 1 - a^2 -1}{h} \\
      & = \underset {h \to  0} \lim \dfrac{2ah+h^2}{h} = \underset {h \to  0} \lim (2a+h) = 2a
\end{align}
$$

We kunnen nu de afgeleide van f in \textit{elk} punt $a$ proberen berekenen. Op die manier verkrijgen we een nieuwe functie die we de afgeleide functie noemen: 

Zij $f(x)$ een reële functie. In de punten waar de afgeleide bestaat definiëren we de **afgeleide functie van $f(x)$**, genoteerd als $f'(x)$, maar ook als ($\dfrac{df}{dx}$ of $\dfrac{d}{dx}f(x)$ of $D f(x)$ of $\dot{f}$). 

We hebben boven gezien dat voor willekeurige $a$, de afgeleide van $f:x\mapsto x^2+1$ in $a$ gelijk is aan $2a$. De afgeleide functie is dus $f':x\mapsto 2x$.

### Rekenregels

### Raaklijn

De *afgeleide* in een punt is dus de *richtingscoëfficiënt* van de raaklijn aan de grafiek in dat punt (Zie voorbeeld \ref{vb:helling_raaklijn}). De vergelijking van de raaklijn aan de grafiek van $f$ in het punt $(a,f(a))$ is dus
		$$
		y-f(a) = f'(a)(x-a)
		$$